In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

In [ ]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [1]:
from envs.features_extractor import ResnetEncoder
from envs.dataset import PhotoEnhancementDataset
dataset = PhotoEnhancementDataset()
encoder = ResnetEncoder()

Using cache found in /home/zakaria/.cache/torch/hub/pytorch_vision_v0.10.0
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
